In [ ]:
import torch
import torchvision
from torchvision.models.detection import MaskRCNN_ResNet50_FPN_Weights
import cv2

In [ ]:
model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT)
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)

In [ ]:
predictions

In [ ]:
import pyrealsense2 as rs
import numpy as np
## define path to bag
bag_path = '/home/richard/realsense_workshop/day_1_pm/resources/table2.bag'

# Create pipeline
pipeline = rs.pipeline()
# Create a config object
config = rs.config()
# Tell config that we will use a recorded device from file to be used by the pipeline through playback.
rs.config.enable_device_from_file(config, bag_path)

# pipeline_wrapper = rs.pipeline_wrapper(pipeline)
# pipeline_profile = config.resolve(pipeline_wrapper)
# Configure the pipeline to stream the depth stream
# Change this parameters according to the recorded bag file resolution
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 15)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 15)

# pipeline would not resolve if the config does not match or if the config is not possible
profile = pipeline.start(config)

depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()

# depth_profile = profile.get_stream(rs.stream.depth)
depth_profile = rs.video_stream_profile(profile.get_stream(rs.stream.depth))
intr = depth_profile.get_intrinsics()
# the depth scale is the multiplier to the number that the 

align_to = rs.stream.color
align = rs.align(align_to)

# Create opencv window to render image in

# Create colorizer object
colorizer = rs.colorizer()

pc = rs.pointcloud()


In [ ]:
def get_3d_bounding_box():
    pass
    

In [ ]:
frames = pipeline.wait_for_frames()
# frames.get_depth_frame() is a 640x360 depth image

# Align the depth frame to color frame
aligned_frames = align.process(frames)

# Get aligned frames
aligned_depth_frame = aligned_frames.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
color_frame = aligned_frames.get_color_frame()

# Validate that both frames are valid
if not aligned_depth_frame or not color_frame:
    pass


depth_intrinsics = rs.video_stream_profile(aligned_depth_frame.profile).get_intrinsics()

depth_image = np.asarray(aligned_depth_frame.get_data())
color_image = np.asarray(color_frame.get_data())

pc.map_to(color_frame)
points = pc.calculate(aligned_depth_frame)


In [ ]:
v, t = points.get_vertices(), points.get_texture_coordinates()
verts = np.asanyarray(v).view(np.float32).reshape(480, 640, 3)  # xyz
texcoords = np.asanyarray(t).view(np.float32).reshape(480, 640, 2)  # uv

In [ ]:
pipeline.stop()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from PIL import Image
from torchvision import transforms as T
import numpy as np

In [ ]:
plt.imshow(color_image)

In [ ]:
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]
# I will link the notebook in the description
# You can copy the class names from the description
# or the notebook
len(COCO_INSTANCE_CATEGORY_NAMES) # 91 classes including background

In [ ]:
def get_prediction(img, threshold=0.5):
  transform = T.Compose([T.ToTensor()]) # Turn the image into a torch.tensor
  img = transform(img)
  # img = img.cuda() # Only if GPU, otherwise comment this line
  pred = model([img]) # Send the image to the model. This runs on CPU, so its going to take time
  #Let's change it to GPU
  # pred = pred.cpu() # We will just send predictions back to CPU
  # Now we need to extract the bounding boxes and masks
  pred_score = list(pred[0]['scores'].detach().cpu().numpy())
  pred_t = [pred_score.index(x) for x in pred_score if x > threshold][-1]
  masks = (pred[0]['masks'] > 0.5).squeeze().detach().cpu().numpy()
  pred_class = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in list(pred[0]['labels'].cpu().numpy())]
  pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in list(pred[0]['boxes'].detach().cpu().numpy())]
  masks = masks[:pred_t+1]
  pred_boxes = pred_boxes[:pred_t+1]
  pred_class = pred_class[:pred_t+1]
  return masks, pred_boxes, pred_class

In [ ]:
# for generating random color for the masks

import random

def random_color_masks(image):
  # I will copy a list of colors here
  colors = [[0, 255, 0],[0, 0, 255],[255, 0, 0],[0, 255, 255],[255, 255, 0],[255, 0, 255],[80, 70, 180], [250, 80, 190],[245, 145, 50],[70, 150, 250],[50, 190, 190]]
  r = np.zeros_like(image).astype(np.uint8)
  g = np.zeros_like(image).astype(np.uint8)
  b = np.zeros_like(image).astype(np.uint8)
  r[image==1], g[image==1], b[image==1] = colors[random.randrange(0, 10)]
  colored_mask = np.stack([r,g,b], axis=2)
  return colored_mask

In [ ]:
def instance_segmentation(img, threshold=0.5, rect_th=3,
                          text_size=1, text_th=1, url=False):
  masks, boxes, pred_cls = get_prediction(img, threshold=threshold)
    
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # For working with RGB images instead of BGR
  for i in range(len(masks)):
    rgb_mask = random_color_masks(masks[i])
    img = cv2.addWeighted(img, 1, rgb_mask, 0.5, 0)
    pt1 = tuple(int(x) for x in boxes[i][0])
    pt2 = tuple(int(x) for x in boxes[i][1])
    cv2.rectangle(img, pt1, pt2, color=(0, 255, 0), thickness=rect_th)
    cv2.putText(img, pred_cls[i], pt1, cv2.FONT_HERSHEY_SIMPLEX, text_size, (0, 255, 0), thickness=text_th)
  return img, pred_cls, masks, boxes

In [ ]:
img, pred_classes, masks, boxes = instance_segmentation(color_image, rect_th=1, text_th=1)

In [ ]:
plt.imshow(img)

In [ ]:
boxes

In [ ]:
#check the predicted classes
pred_classes

In [ ]:
def get_dim_high(points, dim):
    if len(points.shape) ==3:
        tmp = points[:,:,dim].flatten()
    elif len(points.shape) ==2:
        tmp = points[:,dim].flatten()
    tmp = tmp[tmp!=0.0]
    tmp = np.sort(tmp)
    n = tmp[::-1]
    n_bottom = n[0:10].mean()
    return n_bottom

def get_dim_low(points, dim):
    if len(points.shape) ==3:
        tmp = points[:,:,dim].flatten()
    elif len(points.shape) ==2:
        tmp = points[:,dim].flatten()
    tmp = tmp[tmp!=0.0]
    tmp = np.sort(tmp)
    n_top = tmp[0:10].mean()
    return n_top

In [ ]:
def get_bounds(points):
    
    x_l = get_dim_low(points,0)
    y_l = get_dim_low(points,1)
    z_l = get_dim_low(points,2)
    x_h = get_dim_high(points,0)
    y_h = get_dim_high(points,1)
    z_h = get_dim_high(points,2)
    return x_l, y_l, z_l, x_h, y_h, z_h
    

In [ ]:
img_points = verts[masks[0],:]

In [ ]:
x_l, y_l, z_l, x_h, y_h, z_h = get_bounds(img_points)

In [ ]:
img_points

In [ ]:
intrinsics = depth_intrinsics

# pt000 = np.array([x_l, y_l, z_l])
# pt001 = np.array([x_l, y_l, z_h])
# pt010 = np.array([x_l, y_h, z_l])
# pt011 = np.array([x_l, y_h, z_h])
# pt100 = np.array([x_h, y_l, z_h])
# pt101 = np.array([x_h, y_h, z_l])
# pt110 = np.array([x_h, y_h, z_l])
# pt111 = np.array([x_h, y_h, z_h])

pt000 = np.array([x_l, y_l, z_l])
pt100 = np.array([x_l, y_l, z_h])
pt010 = np.array([x_l, y_h, z_l])
pt110 = np.array([x_l, y_h, z_h])
pt001 = np.array([x_h, y_l, z_l])
pt101 = np.array([x_h, y_l, z_h])
pt011 = np.array([x_h, y_h, z_l])
pt111 = np.array([x_h, y_h, z_h])

pixel000 = rs.rs2_project_point_to_pixel(intrinsics,pt000)
pixel001 = rs.rs2_project_point_to_pixel(intrinsics,pt001)
pixel010 = rs.rs2_project_point_to_pixel(intrinsics,pt010)
pixel011 = rs.rs2_project_point_to_pixel(intrinsics,pt011)
pixel100 = rs.rs2_project_point_to_pixel(intrinsics,pt100)
pixel101 = rs.rs2_project_point_to_pixel(intrinsics,pt101)
pixel110 = rs.rs2_project_point_to_pixel(intrinsics,pt110)
pixel111 = rs.rs2_project_point_to_pixel(intrinsics,pt111)

pixel000 = int(pixel000[0]) , int(pixel000[1])
pixel001 = int(pixel001[0]) , int(pixel001[1])
pixel010 = int(pixel010[0]) , int(pixel010[1])
pixel011 = int(pixel011[0]) , int(pixel011[1])
pixel100 = int(pixel100[0]) , int(pixel100[1])
pixel101 = int(pixel101[0]) , int(pixel101[1])
pixel110 = int(pixel110[0]) , int(pixel110[1])
pixel111 = int(pixel111[0]) , int(pixel111[1])

image_with_line = img.copy()
image_with_line = cv2.line(image_with_line,pixel000,pixel001,(0,255,255))
image_with_line = cv2.line(image_with_line,pixel000,pixel010,(0,255,255))
image_with_line = cv2.line(image_with_line,pixel011,pixel001,(0,255,255))
image_with_line = cv2.line(image_with_line,pixel011,pixel010,(0,255,255))

image_with_line = cv2.line(image_with_line,pixel100,pixel101,(255,0,255))
image_with_line = cv2.line(image_with_line,pixel100,pixel110,(255,0,255))
image_with_line = cv2.line(image_with_line,pixel111,pixel101,(255,0,255))
image_with_line = cv2.line(image_with_line,pixel111,pixel110,(255,0,255))

image_with_line = cv2.line(image_with_line,pixel000,pixel100,(255,255,0))
image_with_line = cv2.line(image_with_line,pixel001,pixel101,(255,255,0))
image_with_line = cv2.line(image_with_line,pixel010,pixel110,(255,255,0))
image_with_line = cv2.line(image_with_line,pixel011,pixel111,(255,255,0))



In [ ]:
print(pixel000)
print(pixel001)
print(pixel010)
print(pixel011)
print(pixel100)
print(pixel101)
print(pixel110)
print(pixel111)

In [ ]:
# image_with_line = cv2.line(img,pixel000,pixel100,(255,255,255))
pred_classes

In [ ]:
plt.imshow(image_with_line)

In [ ]:
def draw_lines_from_bounds(img_out, intrin, x_l, y_l, z_l, x_h, y_h, z_h):
    pt000 = np.array(x_l, y_l, z_l)
    pt001 = np.array(x_l, y_l, z_h)
    pt010 = np.array(x_l, y_h, z_l)
    pt011 = np.array(x_l, y_h, z_h)
    pt100 = np.array(x_h, y_l, z_h)
    pt101 = np.array(x_h, y_h, z_l)
    pt110 = np.array(x_h, y_h, z_l)
    pt111 = np.array(x_h, y_h, z_h)

    pixel000 = rs.rs2_project_point_to_pixel(intrin,pt000)
    pixel001 = rs.rs2_project_point_to_pixel(intrin,pt001)
    pixel010 = rs.rs2_project_point_to_pixel(intrin,pt010)
    pixel011 = rs.rs2_project_point_to_pixel(intrin,pt011)
    pixel100 = rs.rs2_project_point_to_pixel(intrin,pt100)
    pixel101 = rs.rs2_project_point_to_pixel(intrin,pt101)
    pixel110 = rs.rs2_project_point_to_pixel(intrin,pt110)
    pixel111 = rs.rs2_project_point_to_pixel(intrin,pt111)

    # draw 12 lines of the cubes


In [ ]:
print(f'X: {x_l} {x_h}')
print(f'Y: {y_l} {y_h}')
print(f'Z: {z_l} {z_h}')

In [ ]:
print(f'X:  {x_h - x_l}')
print(f'Y:  {y_h - y_l}')
print(f'Z:  {z_h - z_l}')

table_z

In [ ]:
img_points = verts[masks[0],:]pc.map_to(color_frame)

In [ ]:
img_points.shape()

rs.rs2

In [ ]:
get_z_top(img_points)

In [ ]:
img_points[:,2].max()

In [ ]:
plt.imshow(img)

In [ ]:
depth_image